In [1]:
# Instalação e Importação de Bibliotecas

!pip install pandas scikit-learn tensorflow numpy

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf

In [2]:
# Carregando o Dataset MovieLens 100k

url = "https://files.grouplens.org/datasets/movielens/ml-100k/u.data"
colunas = ["usuario", "filme", "avaliacao", "tempo"]
dados = pd.read_csv(url, sep='\t', names=colunas)

dados.head()

,usuario,filme,avaliacao,tempo
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
# Preparando os Dados

# Criar uma tabela de usuários x filmes
matriz_avaliacoes = dados.pivot_table(index='usuario', columns='filme', values='avaliacao')

# Divide em treino e teste (80% e 20%)
train, test = train_test_split(dados, test_size=0.2, random_state=42)

In [6]:
# Filtragem Colaborativa Baseada em Usuário

# Calcula a similaridade entre os usuários
similaridade_usuarios = cosine_similarity(matriz_avaliacoes.fillna(0))
sim_usuarios_df = pd.DataFrame(
    similaridade_usuarios,
    index=matriz_avaliacoes.index,
    columns=matriz_avaliacoes.index
)

# Função para prever a nota que um usuário daria a um filme
def prever_avaliacao_usuario(usuario, filme):
    # Caso o filme não exista na matriz
    if filme not in matriz_avaliacoes.columns:
        return 3.0  # nota média neutra padrão

    # Pega as avaliações dos usuários para esse filme
    notas_usuario = matriz_avaliacoes[filme]

    # Pega a similaridade do usuário com os outros
    similaridade = sim_usuarios_df[usuario]

    # Considera apenas os usuários que avaliaram o filme
    mask = notas_usuario.notna()

    # Se ninguém avaliou o filme, retorna valor neutro
    if mask.sum() == 0:
        return 3.0

    pred = np.dot(similaridade[mask], notas_usuario[mask]) / similaridade[mask].sum()
    return pred

# Avaliação no conjunto de teste
y_true, y_pred = [], []

for _, row in test.iterrows():
    y_true.append(row['avaliacao'])
    y_pred.append(prever_avaliacao_usuario(row['usuario'], row['filme']))

# Cálculo das métricas
mse_user = mean_squared_error(y_true, y_pred)
rmse_user = np.sqrt(mse_user)
mae_user = mean_absolute_error(y_true, y_pred)

print(f"Filtragem Colaborativa (Usuário) — RMSE: {rmse_user:.3f}, MAE: {mae_user:.3f}")


Filtragem Colaborativa (Usuário) — RMSE: 0.955, MAE: 0.760


In [8]:
# Filtragem Colaborativa Baseada em Itens

# Calcula a similaridade entre os filmes
similaridade_itens = cosine_similarity(matriz_avaliacoes.fillna(0).T)
sim_itens_df = pd.DataFrame(
    similaridade_itens,
    index=matriz_avaliacoes.columns,
    columns=matriz_avaliacoes.columns
)

# Função para prever a nota que um usuário daria a um filme
# com base na similaridade entre os itens
def prever_avaliacao_item(usuario, filme):
    # Caso o filme não exista na matriz
    if filme not in matriz_avaliacoes.columns:
        return 3.0  # nota média neutra padrão

    # Pega todas as notas que o usuário deu
    notas_usuario = matriz_avaliacoes.loc[usuario]

    # Pega a similaridade do filme atual com os outros filmes
    similaridade = sim_itens_df[filme]

    # Considera apenas os filmes avaliados pelo usuário
    mask = notas_usuario.notna()

    # Se o usuário não avaliou nada, retorna nota neutra
    if mask.sum() == 0:
        return 3.0

    pred = np.dot(similaridade[mask], notas_usuario[mask]) / similaridade[mask].sum()
    return pred

# Avaliação no conjunto de teste
y_true, y_pred = [], []

for _, row in test.iterrows():
    y_true.append(row['avaliacao'])
    y_pred.append(prever_avaliacao_item(row['usuario'], row['filme']))

# Cálculo das métricas
mse_item = mean_squared_error(y_true, y_pred)
rmse_item = np.sqrt(mse_item)
mae_item = mean_absolute_error(y_true, y_pred)

print(f"Filtragem Colaborativa (Item) — RMSE: {rmse_item:.3f}, MAE: {mae_item:.3f}")


Filtragem Colaborativa (Item) — RMSE: 0.975, MAE: 0.777


In [10]:
# Autoencoder (Aprendizado Profundo)

# Substitui valores ausentes por 0
matriz_treino = matriz_avaliacoes.fillna(0).values

# Define o modelo de Autoencoder
entrada = tf.keras.layers.Input(shape=(matriz_treino.shape[1],))
codificador = tf.keras.layers.Dense(64, activation='relu')(entrada)
decodificador = tf.keras.layers.Dense(matriz_treino.shape[1], activation='linear')(codificador)

modelo = tf.keras.Model(entrada, decodificador)
modelo.compile(optimizer='adam', loss='mse')

# Treinamento do modelo
modelo.fit(matriz_treino, matriz_treino, epochs=5, batch_size=32, verbose=1)

# Predições e avaliação
predicoes = modelo.predict(matriz_treino)

# Cálculo das métricas
mse_auto = mean_squared_error(matriz_treino.flatten(), predicoes.flatten())
rmse_auto = np.sqrt(mse_auto)
mae_auto = mean_absolute_error(matriz_treino.flatten(), predicoes.flatten())

print(f"Autoencoder — RMSE: {rmse_auto:.3f}, MAE: {mae_auto:.3f}")


Epoch 1/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.8266
Epoch 2/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.6135
Epoch 3/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.5577
Epoch 4/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.4997
Epoch 5/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.4726
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Autoencoder — RMSE: 0.680, MAE: 0.313


In [11]:
# Comparação Final

resultados = pd.DataFrame({
'Método': ['Colab. Usuário', 'Colab. Item', 'Autoencoder'],
'RMSE': [rmse_user, rmse_item, rmse_auto],
'MAE': [mae_user, mae_item, mae_auto]
})
print(resultados)

           Método      RMSE       MAE
0  Colab. Usuário  0.954667  0.759921
1     Colab. Item  0.974673  0.776873
2     Autoencoder  0.680396  0.313382


In [12]:
# PERGUNTA: Qual abordagem foi mais eficiente na recomendação de filmes? Como melhorar o
# sistema de recomendação?

# Qual abordagem foi mais eficiente na recomendação de filmes?
#- O Autoencoder Denso foi a abordagem mais eficiente. Ele apresentou RMSE = 0.680 e MAE = 0.313,
# valores significativamente melhores que os métodos de filtragem colaborativa.
# Isso mostra que suas previsões ficaram mais próximas das avaliações reais dos usuários.

# Como melhorar o sistema de recomendação?
#- Testar com novos dados.
#- Considerar modelos híbridos (autoencoder + filtragem colaborativa).
#- Incorporar metadados de usuários e filmes para enriquecer recomendações.